In [1]:
import tensorflow as tf
import numpy as np
import os
import tqdm
from scipy.io import savemat
from scipy.io import loadmat
import cv2

tf.random.set_seed(42)

train_set = 'mnist'
mode = 'grayscale'

if mode == 'color':
    input_shape = (32, 32, 3)
    datasets = [
        'svhn_cropped',
        'cifar10',
        'celeb_a',
        'gtsrb',
        'compcars',
        'lsun',
        'noise',
        'constant'
    ]
elif mode == 'grayscale':
    input_shape = (32, 32, 1)
    datasets = [
        'mnist',
        'fashion_mnist',
        'emnist_letters',
        'sign_lang',
        'clevr',
        'noise',
        'constant'
    ]
    
probs = {}

# load log probabilities
probs_ll = loadmat('../probs/' + train_set + '.mat')

def find_png_size(x):
    # returns png size for a batch of images
    x = x.astype('int32')
    tmp = []
    for i in range (x.shape[0]):
        tmp.append(len(cv2.imencode('.png', np.squeeze(x[i,:,:,:]), [int(cv2.IMWRITE_PNG_COMPRESSION), 9])[1])*8)
    return tmp

def find_jpeg_size(x):
    # returns jpeg size for a batch of images
    x = x.astype('int32')
    tmp = []
    for i in range (x.shape[0]):
        cv2.imwrite('./temp.png', np.squeeze(x[i,:,:,:]))
        !magick ./temp.png -quality 0 ./temp.jp2 &> out.log
        tmp.append(os.path.getsize('./temp.jp2')*8)
        !rm ./temp.jp2
    return tmp

def find_flif_size(x):
    # returns flif size for a batch of images
    x = x.astype('int32')
    tmp = []
    for i in range (x.shape[0]):
        cv2.imwrite('./temp.png', np.squeeze(x[i,:,:,:]))
        !flif -e -o ./temp.png ./temp.flif &> out.log
        tmp.append(os.path.getsize('./temp.flif')*8)
    return tmp
    
for dataset in datasets:

    _, _, ds_test = dataset_utils.get_dataset(
          dataset,
          512,
          mode,
          mutation_rate=0
      )
    
    tmp = []
    
    # get compressed lengths for the test samples
    for test_batch in tqdm.tqdm(ds_test):
        tmp.append(find_jpeg_size(test_batch.numpy()))

    tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)
    probs[dataset+'_size'] = tmp
    
    # compute IC score
    probs[dataset] = -(probs_ll[dataset]/np.log(2)) - probs[dataset+'_size']

save_dir = '../probs_ic_jpeg/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

savemat(save_dir + train_set + '.mat', probs)